In [7]:
import requests
import re
from bs4 import BeautifulSoup
import html5lib
import multiprocessing.dummy as mp

numInPage = 250
URL = f'https://afbulletsafe.com/?PageSize={numInPage}&SortOption=6&PageNumber=1'
ROOT = 'http://www.afbulletsafe.com/'
outdir = '../scrapes/'
outfile = outdir + 'afbulletsafe.txt'
homepage = requests.get(URL)

# need HTML5LIB because sometimes website has errors in HTML. html5lib is very forgiving.
homeSoup = BeautifulSoup(homepage.content,'html5lib')


In [9]:
# Manually calculating number of pages based on web scrape result of total results
#  and generating URLS using that
totalStr = homeSoup.find(string=re.compile('per page of (\d+)')).strip()
total = int(re.match('per page of (\d+)', totalStr).groups()[0])
totalPages = int(total/numInPage)+1
URLs = lambda pageNum: f'https://afbulletsafe.com/?PageSize={numInPage}&SortOption=6&PageNumber={pageNum}'
pageURLs = [URLs(page) for page in range(1,totalPages+1)]

In [11]:
def getBulletsInPage(pageURL, bulletDict):
    
    page = requests.get(pageURL)
    pageSoup = BeautifulSoup(page.content, 'html5lib')

    # Searches for all strings that match regex corresponding to a bullet link
    bulletScrape = pageSoup.find_all('a',href=re.compile('/Bullet/Details/\d+/'))
    bulletDict[pageURL] = [bullet.get_text().strip() for bullet in bulletScrape]


In [13]:
# multiprocessing is used to speed up the scrape. 

# a dict/manager is used to collect all scraped bullets during multiprocessing.  
manager = mp.Manager()
resultsDict = manager.dict()

#running Pool() with no arguments will default to amount of cores you have... I think
p = mp.Pool()

p.starmap(getBulletsInPage, [(pageURL,resultsDict) for pageURL in pageURLs])
p.close()
p.join()

# Writing to file. Should default to clobbering.
with open(outfile,'w',encoding='utf-8') as fileID:
    for key in resultsDict:
        fileID.write(f'SECTION {key} \n')
        for line in resultsDict[key]:
            fileID.write(line + '\n')

In [15]:
# this block is for debugging
pageURL = 'https://afbulletsafe.com/?PageSize=250&SortOption=6&PageNumber=24'
page = requests.get(pageURL)
pageSoup = BeautifulSoup(page.content, 'html5lib')
bulletScrape = pageSoup.find_all('a',href=re.compile('/Bullet/Details/\d+/'))
bullets = [bullet.get_text() for bullet in bulletScrape]
bullets[0:5]

[&#39;Moved Op Ice Bridge cgo; loaded 5 pcs/4 tons--helped bridge gap/support observation of polar cap&#39;,
 &#39;Assembled/ rushed 3K ration pallets valued &gt;$6M--supplied life-saving perishables to combat engaged servicemen&#39;,
 &#39;Pivotal Latrine Svc Truck trainer; provided tng dur 183% increase of pipeline students--afforded comfort for xxK pax&#39;,
 &#39;Sped movement of (TYPE) acft; processed 2K lb turbofan engine--delivered $XXM (ITEM) to FMC status&#39;,
 &#39;Delivered critical (ITEM) eqpt; mvd hi-vis global security shipment/lifted w/in 4 hrs--sustained $000M (ACRONYM) sys&#39;]